# Notebook to apply hillshade model correction to Planet images at Wolverine Glacier, AK

GEOPH 520: DSP

Rainey Aberle

April 2022


### Outline

1. Initial setup:
    - Import packages
    - Define paths in directory
2. Load WorldView-derived DEM and Planet images
3. Loop through images:
    - Create hillshade model
    - Resample hillshade to image coordinates
    - Determine optimal band scalars for correction, apply correction to image

## 1. Initial Setup

In [ ]:
# -----Import packages
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import time
import rasterio as rio
import geopandas as gpd
from pyproj import Proj, transform, Transformer
from functools import partial
from functions import sunpos
from shapely.geometry import Polygon
import subprocess
from scipy.interpolate import interp2d

In [ ]:
# -----Define paths in directory
# path to DEM
DEM_path = '/Users/raineyaberle/Research/PhD/Wolverine/DEMs/WG_20200728-DEM_mosaic_crop_UTM6_resamp_filled.tif'
# path to Planet images
im_path = '/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/PSScene4Band/'
# path to hillshades directory
hs_path = im_path+'hillshades/'
# output folder
out_path = im_path+'../'
# test results output folder (if outputting all scalar tests)
results_path = out_path+'hillshade-correction-tests/'
# path to Area of Interest (AOI) shapefile, e.g. glacier outline (optional)
AOI_path = '/Users/raineyaberle/Research/PhD/Wolverine/GIS_data/wolverine_RGI.shp'

# -----Determine whether you would like outputs plotted
plot_results = 1 # = 1 to plot

## 2. Define snow-covered area, load WorldView-derived DEM and Planet images

In [ ]:
# -----Define area of interest (AOI) 
# Used to minimize standard deviation in hillshade model correction. 
# Choose a region that where the surface reflectance in each image should be nearly uniform, 
#     but may be impacted by shadows. 
AOI = Polygon([(394e3, 6698.5e3), (394e3, 6700.5e3), (396e3, 6700.5e3), (396e3, 6698.5e3), (394e3, 6698.5e3)])

# -----Define CRS (EPSG code)
crs = 32606

# -----Load DEM
DEM = rio.open(DEM_path)
# coordinates
DEM_x = np.linspace(DEM.bounds.left, DEM.bounds.right, num=np.shape(DEM.read(1))[1])
DEM_y = np.linspace(DEM.bounds.top, DEM.bounds.bottom, num=np.shape(DEM.read(1))[0])

# -----Load images
ims = os.chdir(im_path) # change directory
im_names = glob.glob('*SR_clip.tif') # load all files with SR_clip.tif extension
im_names.sort() # sort file names by date
# print image file names
# print('Image names from file:')
# print(im_names)

# -----Print coordinate reference systems to ensure they are the same
print('DEM CRS: ',DEM.crs)
print('Images CRS:',rio.open(im_names[0]).crs)

## 3. Loop through images

In [ ]:
# start time
t1 = time.monotonic()

# loop through images
for im_name in im_names:
    
    # -----Load image
    print('Image: ',im_name)
    im = rio.open(im_name)
    # load bands (blue, green, red, near infrared)
    im_scalar = 10000
    b = im.read(1).astype(float) / im_scalar
    g = im.read(2).astype(float) / im_scalar
    r = im.read(3).astype(float) / im_scalar
    nir = im.read(4).astype(float) / im_scalar 
    # define coordinates
    im_x = np.linspace(im.bounds.left, im.bounds.right, num=np.shape(b)[1])
    im_y = np.linspace(im.bounds.top, im.bounds.bottom, num=np.shape(b)[0])

    # -----Extract image information for sun position calculation
    # location: grab center image coordinate, convert to lat lon
    xmid = ((im.bounds.right - im.bounds.left)/2 + im.bounds.left)
    ymid = ((im.bounds.top - im.bounds.bottom)/2 + im.bounds.bottom)
    transformer = Transformer.from_crs("epsg:"+str(crs), "epsg:4326")
    location = transformer.transform(xmid, ymid)
    # when: year, month, day, hour, minute, second
    when = (float(im_name[0:4]), float(im_name[4:6]), float(im_name[6:8]), 
            float(im_name[9:11]), float(im_name[11:13]), float(im_name[13:15]))
    # sun azimuth and elevation
    azimuth, elevation = sunpos(when, location, refraction=1)
    print('    location:',location)
    print('    when:',when)
    print('    azimuth: ',azimuth)
    print('    elevation:',elevation)

    # -----Create hillshade model (if it does not already exist in file)
    hs_fn = out_path+'hillshades/'+str(azimuth)+'-az_'+str(elevation)+'-z_hillshade.tif'
    if os.path.exists(hs_fn):
        print('    hillshade model already exists in directory... loading')
    else:
        print('    creating hillshade model...')
        # construct the gdal_merge command
        # modified from: https://github.com/clhenrick/gdal_hillshade_tutorial
        # gdaldem hillshade -az aximuth -z elevation dem.tif hillshade.tif
        cmd = 'gdaldem hillshade -az '+str(azimuth)+' -z '+str(elevation)+' '+str(DEM_path)+' '+hs_fn
        # run the command 
        p = subprocess.run(cmd, shell=True, capture_output=True) 
        print(p)

    # -----load hillshade model from file
    hs = rio.open(hs_fn)
    print('    hillshade model loaded from file')
    # coordinates
    hs_x = np.linspace(hs.bounds.left, hs.bounds.right, num=np.shape(hs.read(1))[1])
    hs_y = np.linspace(hs.bounds.top, hs.bounds.bottom, num=np.shape(hs.read(1))[0])

    # -----Resample hillshade to image coordinates
    # resampled hillshade file name
    hs_resamp_fn = out_path+'hillshades/'+str(azimuth)+'-az_'+str(elevation)+'-z_hillshade_resamp.tif'
    # check if file already exists in directory
    if os.path.exists(hs_resamp_fn):
        print('    resampled hillshade model already exists in directory... loading')
    # resample if it doesn't exist
    else:
        print('    resampling hillshade...')
        # create interpolation object
        f = interp2d(hs_x, hs_y, hs.read(1))
        hs_resamp = f(im_x, im_y)
        # save to file
        with rio.open(hs_resamp_fn,'w',
                      driver='GTiff',
                      height=hs_resamp.shape[0],
                      width=hs_resamp.shape[1],
                      dtype=hs_resamp.dtype,
                      count=1,
                      crs=im.crs,
                      transform=im.transform) as dst:
            dst.write(hs_resamp, 1)
        print('    resampled hillshade model saved to file:',hs_resamp_fn)

    # -----load resampled hillshade
    hs_resamp = rio.open(hs_resamp_fn).read(1)
    print('    resampled hillshade model loaded from file')
    
    # -----clip image and hillshade to AOI
    Ix = np.where((im_x >= AOI.bounds[0]) & (im_x <= AOI.bounds[2]))[0]
    Iy = np.where((im_y >= AOI.bounds[1]) & (im_y <= AOI.bounds[3]))[0]
    b_AOI = np.zeros((np.shape(Ix)[0], np.shape(Iy)[0]))
    g_AOI = np.zeros((np.shape(Ix)[0], np.shape(Iy)[0]))
    r_AOI = np.zeros((np.shape(Ix)[0], np.shape(Iy)[0]))
    nir_AOI = np.zeros((np.shape(Ix)[0], np.shape(Iy)[0]))
    hs_AOI = np.zeros((np.shape(Ix)[0], np.shape(Iy)[0]))
    print(Iy,Ix)
    for i in np.arange(0,np.shape(Ix)[0]-1):
        for j in np.arange(0, np.shape(Iy)[0]-1):
            b_AOI[j,i] = b[Iy[j], Ix[i]]
            g_AOI[j,i] = g[Iy[j], Ix[i]]
            r_AOI[j,i] = r[Iy[j], Ix[i]]
            nir_AOI[j,i] = nir[Iy[j], Ix[i]]
            hs_AOI[j,i] = hs_resamp[Iy[j], Ix[i]]
            
    # -----normalize hillshade model
    hs_norm = (hs_resamp_smooth - np.min(hs_resamp_smooth)) / (np.max(hs_resamp_smooth) - np.min(hs_resamp_smooth))
    hs_AOI_norm = (hs_AOI - np.min(hs_AOI)) / (np.max(hs_AOI) - np.min(hs_AOI))
    
    # -----loop through hillshade scalar multipliers
    print('    solving for optimal band scalars')
    # define scalars to test
    hs_scalars = np.linspace(0,0.5,num=21)
    # blue
    b_AOI_mu = [] # mean 
    b_AOI_sigma = [] # std
    # green
    g_AOI_mu = [] # mean 
    g_AOI_sigma = [] # std
    # red
    r_AOI_mu = [] # mean 
    r_AOI_sigma = [] # std
    # nir
    nir_AOI_mu = [] # mean 
    nir_AOI_sigma = [] # std
    for hs_scalar in hs_scalars:
        # full image
        b_adj = b - (hs_norm * hs_scalar)
        g_adj = g - (hs_norm * hs_scalar)
        r_adj = r - (hs_norm * hs_scalar)
        nir_adj = nir - (hs_norm * hs_scalar)
        # AOI
        b_AOI_mu = b_AOI_mu + [np.nanmean(b_AOI - (hs_AOI_norm * hs_scalar))]
        b_AOI_sigma = b_AOI_sigma + [np.nanstd(b_AOI - (hs_AOI_norm * hs_scalar))] 
        g_AOI_mu = g_AOI_mu + [np.nanmean(g_AOI - (hs_AOI_norm * hs_scalar))]
        g_AOI_sigma = g_AOI_sigma + [np.nanstd(g_AOI - (hs_AOI_norm * hs_scalar))]
        r_AOI_mu = r_AOI_mu + [np.nanmean(r_AOI - (hs_AOI_norm * hs_scalar))]
        r_AOI_sigma = r_AOI_sigma + [np.nanstd(r_AOI - (hs_AOI_norm * hs_scalar))]
        nir_AOI_mu = nir_AOI_mu + [np.nanmean(nir_AOI - (hs_AOI_norm * hs_scalar))]
        nir_AOI_sigma = nir_AOI_sigma + [np.nanstd(nir_AOI - (hs_AOI_norm * hs_scalar))]
    
    # -----Determine optimal scalar for each image band
    Ib = np.where(b_AOI_sigma==np.min(b_AOI_sigma))[0][0]
    b_scalar = hs_scalars[Ib]
    Ig = np.where(g_AOI_sigma==np.min(g_AOI_sigma))[0][0]
    g_scalar = hs_scalars[Ig]    
    Ir = np.where(r_AOI_sigma==np.min(r_AOI_sigma))[0][0]
    r_scalar = hs_scalars[Ir]    
    Inir = np.where(nir_AOI_sigma==np.min(nir_AOI_sigma))[0][0]
    nir_scalar = hs_scalars[Inir]
    
    # -----Apply optimal hillshade model correction
    b_corrected = b - (hs_norm * hs_scalars[Ib])
    g_corrected = g - (hs_norm * hs_scalars[Ig])
    r_corrected = r - (hs_norm * hs_scalars[Ir])
    nir_corrected = nir - (hs_norm * hs_scalars[Inir])
    
    # -----Plot original and corrected images and band histograms
    if plot_results==1:
        fig1, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(16,16), gridspec_kw={'height_ratios': [3, 1]})
        plt.rcParams.update({'font.size': 14, 'font.serif': 'Arial'})
        # original image
        ax1.imshow(np.dstack([r, g, b]), 
                   extent=(np.min(im_x)/1000, np.max(im_x)/1000, np.min(im_y)/1000, np.max(im_y)/1000))
        ax1.set_ylabel('Northing [km]')
        ax1.set_ylabel('Easting [km]')
        ax1.set_title('Original image')
        # corrected image
        ax2.imshow(np.dstack([r_corrected, g_corrected, b_corrected]), 
                   extent=(np.min(im_x)/1000, np.max(im_x)/1000, np.min(im_y)/1000, np.max(im_y)/1000))
        ax2.set_ylabel('Easting [km]')
        ax2.set_title('Corrected image')
        # band histograms
        ax3.hist(nir.flatten(), bins=100, color='purple', alpha=0.5, label='NIR')
        ax3.hist(b.flatten(), bins=100, color='blue', alpha=0.5, label='Blue')
        ax3.hist(g.flatten(), bins=100, color='green', alpha=0.5, label='Green')
        ax3.hist(r.flatten(), bins=100, color='red', alpha=0.5, label='Red')
        ax4.hist(nir_corrected.flatten(), bins=100, color='purple', alpha=0.5, label='NIR')
        ax4.hist(b_corrected.flatten(), bins=100, color='blue', alpha=0.5, label='Blue')
        ax4.hist(g_corrected.flatten(), bins=100, color='green', alpha=0.5, label='Green')
        ax4.hist(r_corrected.flatten(), bins=100, color='red', alpha=0.5, label='Red')        
        plt.show()
        
    print('----------')
    print(' ')
    
# end time
t2 = time.monotonic()
print('time elapsed:',str(np.round(t2-t1,2)),'s')